In [ ]:
#| default_exp production.metadata

# production.metadata

> This module includes the metadata calculations which are used to calculate the metadata of the 

In [ ]:
# | export

import logging
from planetarypy.pds.apps import get_index
from planetarypy.hirise import ProductPathfinder
import pandas as pd

from p4tools.production.projection import P4Mosaic

In [ ]:
# | export

logger = logging.getLogger(__name__)

In [ ]:
# | export

class MetadataReader:
    """
    A class to read and manage metadata for a given observation ID (obsid).
    Parameters
    ----------
    obsid : str
        The observation ID for which metadata is to be read and managed.
    Attributes
    ----------
    obsid : str
        The observation ID.
    prodid : ProductPathfinder
        An instance of ProductPathfinder initialized with the observation ID and "_COLOR" suffix.
    labelpath : pathlib.Path
        The local path to the label file.
    label : labels.HiRISE_Label
        An instance of HiRISE_Label initialized with the label path.
    campt_out_path : pathlib.Path
        The path to the campt output CSV file.
    campt_out_df : pandas.DataFrame
        A DataFrame containing the contents of the campt output CSV file.
    Methods
    -------
    read_edr_index()
        Reads the EDR index for the HiRISE instrument.
    download_label()
        Downloads the label file if it does not exist locally.
    """


    def __init__(self, obsid):
        self.obsid = obsid
        self.prodid = ProductPathfinder(obsid+"_COLOR")
        if not self.labelpath.exists():
            self.download_label()

    def read_edr_index(self):
        """Reads the EDR index for the HiRISE instrument"""
        return get_index("mro.hirise", "edr")

    @property
    def labelpath(self):
        """local path to the label file"""
        return self.prodid.local_label_path

    def download_label(self):
        """Downloads the label file if it does not exist locally"""
        self.prodid.download_label()

    @property
    def label(self):
        """HiRISE_Label initialized with the label path"""
        return labels.HiRISE_Label(self.labelpath)

    @property
    def campt_out_path(self):
        """The path to the campt output CSV file"""
        p4m = P4Mosaic(self.obsid)
        return p4m.mosaic_path.parent / f"{self.obsid}_campt_out.csv"

    @property
    def campt_out_df(self):
        """DataFrame containing the contents of the campt output CSV file"""
        return pd.read_csv(self.campt_out_path)


In [ ]:
# | export

def get_north_azimuths_from_SPICE(obsids):
    """
    Calculate the North Azimuth for a list of observation IDs using SPICE metadata.
    Parameters
    ----------
    obsids : list of str
        List of observation IDs for which to calculate the North Azimuth direction.
    Returns
    -------
    pandas.DataFrame
        DataFrame containing the observation IDs and their corresponding North Azimuth values.
        The DataFrame has two columns: 'OBSERVATION_ID' and 'north_azimuth'.
    """
    NAs = []
    for obsid in obsids:
        meta = MetadataReader(obsid)
        NAs.append(meta.campt_out_df['NorthAzimuth'].median())
    return pd.DataFrame(dict(OBSERVATION_ID=obsids, north_azimuth=NAs))